In [2]:
import pandas as pd
import sys
import numpy as np
import warnings
warnings.filterwarnings('ignore')
sys.path.append( '../util' )
import util as util
import matplotlib.pyplot as plt
from ctgan import CTGAN
from sklearn.preprocessing import LabelEncoder
import joblib

/var/folders/1s/tyt3m2l12dz6s028_blqch7r0000gn/T/ipykernel_2639/3191837829.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
###in order to reverse the label encoder, we need to re-fit the label encoder  
###and then reverse the process for the generated data
### there is NO need to run this cell again anymore, we already saved the outputted label encoder.
### if you run it again, don't commit the new encoder.
discrete_columns = [util.y_column, 'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 
                    'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC']

train, test = util.import_dataset(7,"dnn",.5)
encoders = {}

for column in discrete_columns:
    le = LabelEncoder()
    train[column] = le.fit_transform(train[column])
    print(train[column].unique())
    encoders[column] = le
joblib.dump(encoders, 'column_encoders.joblib')
del train,test

Importing datasets.....
Processing training and test data....
Finished processing training and test data
Mapping labels to seven classes.....
Data import and processing complete....
[4 2 3 0 6 1 5 7]
[0 1]
[0 1]
[0 1]
[0]
[0 1]
[0 1]
[0]
[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[1 0]
[1 0]


In [4]:
ctgan_model = CTGAN.load('../generator_custom/medium_synthesizer.pkl')


In [9]:
synthetic_data = ctgan_model.sample(10)
print(synthetic_data)
print(synthetic_data.dtypes)

   flow_duration  Header_Length  Protocol Type  Duration      Rate     Srate  \
0      -0.020576      -0.166253      -0.900942 -0.168614 -0.091585 -0.091639   
1       0.002323      -0.165794      -0.343282 -0.158545 -0.090520 -0.091203   
2      -0.019710      -0.165345      -0.342929 -0.167566 -0.089563 -0.090053   
3      -0.019959      -0.165776      -0.343433 -0.167595 -0.090508 -0.090716   
4      -0.021266      -0.166377      -0.343020 -0.167136 -0.089516 -0.089930   
5      -0.001864      -0.166331      -0.342602 -0.133057 -0.090680 -0.091312   
6      -0.021202       0.010552       0.877858 -0.170083 -0.091679 -0.090974   
7      -0.020739      -0.165465      -0.342786 -0.166744 -0.090761 -0.090722   
8      -0.020330      -0.166184      -0.342686 -0.168274 -0.090563 -0.090432   
9      -0.020270      -0.165815      -0.342462 -0.168667 -0.089847 -0.090299   

      Drate  fin_flag_number  syn_flag_number  rst_flag_number  ...       Std  \
0 -0.000631        -0.307751        -0

In [10]:
encoders_new = joblib.load('column_encoders.joblib')
for column in discrete_columns:
    if column in encoders:
        #####for doing this for the dnn's,do NOT do this for the label column######
        synthetic_data[column] = encoders[column].inverse_transform(synthetic_data[column])
synthetic_data[util.y_column] = synthetic_data[util.y_column].astype('category')
print(synthetic_data.dtypes)
print(synthetic_data)

flow_duration       float32
Header_Length       float32
Protocol Type       float32
Duration            float32
Rate                float32
Srate               float32
Drate               float32
fin_flag_number     float32
syn_flag_number     float32
rst_flag_number     float32
psh_flag_number     float32
ack_flag_number     float32
ece_flag_number     float32
cwr_flag_number     float32
ack_count           float32
syn_count           float32
fin_count           float32
urg_count           float32
rst_count           float32
HTTP                float64
HTTPS               float64
DNS                 float64
Telnet              float64
SMTP                float64
SSH                 float64
IRC                 float64
TCP                 float64
UDP                 float64
DHCP                float64
ARP                 float64
ICMP                float64
IPv                 float64
LLC                 float64
Tot sum             float32
Min                 float32
Max                 